This Jupyter Notebook downloads census tract or block group shape files from U.S. Census API for the specified state  
You only need to enter the desired state's name or abbreviation, intended geographic level (Census Tract or Block Group), year, and run the code.

In [ ]:
### input parameters ###
year = int(2019)
state = "mn"
geo_level = "ct"

In [ ]:
#### insert required libraries ####
import numpy as np
import pandas as pd
import sys
import optparse
import os
import sys
import zipfile
from os.path import isdir, join, normpath

In [ ]:
DOWNLOAD_DIR = 'downloaded_files'
EXTRACT_DIR = 'extracted_files'

# make sure we have the expected directories
for path in [DOWNLOAD_DIR, EXTRACT_DIR]:
    if not isdir(path):
        os.makedirs(path)

In [ ]:
STATE_FIPS_DICT = {
    '01': {
        'abbreviation': 'AL', 'name': 'Alabama',
        'region': 3, 'region_name': 'South',
        'division': 6, 'division_name': 'East South Central',
    },
    '02': {
        'abbreviation': 'AK', 'name': 'Alaska',
        'region': 4, 'region_name': 'West',
        'division': 9, 'division_name': 'Pacific',
    },
    '04': {
        'abbreviation': 'AZ', 'name': 'Arizona',
        'region': 4, 'region_name': 'West',
        'division': 8, 'division_name': 'Mountain',
    },
    '05': {
        'abbreviation': 'AR', 'name': 'Arkansas',
        'region': 3, 'region_name': 'South',
        'division': 7, 'division_name': 'West South Central',
    },
    '06': {
        'abbreviation': 'CA', 'name': 'California',
        'region': 4, 'region_name': 'West',
        'division': 9, 'division_name': 'Pacific',
    },
    '08': {
        'abbreviation': 'CO', 'name': 'Colorado',
        'region': 4, 'region_name': 'West',
        'division': 8, 'division_name': 'Mountain',
    },
    '09': {
        'abbreviation': 'CT', 'name': 'Connecticut',
        'region': 1, 'region_name': 'Northeast',
        'division': 1, 'division_name': 'New England',
    },
    '10': {
        'abbreviation': 'DE', 'name': 'Delaware',
        'region': 3, 'region_name': 'South',
        'division': 5, 'division_name': 'South Atlantic',
    },
    '11': {
        'abbreviation': 'DC', 'name': 'District of Columbia',
        'region': 3, 'region_name': 'South',
        'division': 5, 'division_name': 'South Atlantic',
    },
    '12': {
        'abbreviation': 'FL', 'name': 'Florida',
        'region': 3, 'region_name': 'South',
        'division': 5, 'division_name': 'South Atlantic',
    },
    '13': {
        'abbreviation': 'GA', 'name': 'Georgia',
        'region': 3, 'region_name': 'South',
        'division': 5, 'division_name': 'South Atlantic',
    },
    '15': {
        'abbreviation': 'HI', 'name': 'Hawaii',
        'region': 4, 'region_name': 'West',
        'division': 9, 'division_name': 'Pacific',
    },
    '16': {
        'abbreviation': 'ID', 'name': 'Idaho',
        'region': 4, 'region_name': 'West',
        'division': 8, 'division_name': 'Mountain',
    },
    '17': {
        'abbreviation': 'IL', 'name': 'Illinois',
        'region': 2, 'region_name': 'Midwest',
        'division': 3, 'division_name': 'East North Central',
    },
    '18': {
        'abbreviation': 'IN', 'name': 'Indiana',
        'region': 2, 'region_name': 'Midwest',
        'division': 3, 'division_name': 'East North Central',
    },
    '19': {
        'abbreviation': 'IA', 'name': 'Iowa',
        'region': 2, 'region_name': 'Midwest',
        'division': 4, 'division_name': 'West North Central',
    },
    '20': {
        'abbreviation': 'KS', 'name': 'Kansas',
        'region': 2, 'region_name': 'Midwest',
        'division': 4, 'division_name': 'West North Central',
    },
    '21': {
        'abbreviation': 'KY', 'name': 'Kentucky',
        'region': 3, 'region_name': 'South',
        'division': 6, 'division_name': 'East South Central',
    },
    '22': {
        'abbreviation': 'LA', 'name': 'Louisiana',
        'region': 3, 'region_name': 'South',
        'division': 7, 'division_name': 'West South Central',
    },
    '23': {
        'abbreviation': 'ME', 'name': 'Maine',
        'region': 1, 'region_name': 'Northeast',
        'division': 1, 'division_name': 'New England',
    },
    '24': {
        'abbreviation': 'MD', 'name': 'Maryland',
        'region': 3, 'region_name': 'South',
        'division': 5, 'division_name': 'South Atlantic',
    },
    '25': {
        'abbreviation': 'MA', 'name': 'Massachusetts',
        'region': 1, 'region_name': 'Northeast',
        'division': 1, 'division_name': 'New England',
    },
    '26': {
        'abbreviation': 'MI', 'name': 'Michigan',
        'region': 2, 'region_name': 'Midwest',
        'division': 3, 'division_name': 'East North Central',
    },
    '27': {
        'abbreviation': 'MN', 'name': 'Minnesota',
        'region': 2, 'region_name': 'Midwest',
        'division': 4, 'division_name': 'West North Central',
    },
    '28': {
        'abbreviation': 'MS', 'name': 'Mississippi',
        'region': 3, 'region_name': 'South',
        'division': 6, 'division_name': 'East South Central',
    },
    '29': {
        'abbreviation': 'MO', 'name': 'Missouri',
        'region': 2, 'region_name': 'Midwest',
        'division': 4, 'division_name': 'West North Central',
    },
    '30': {
        'abbreviation': 'MT', 'name': 'Montana',
        'region': 4, 'region_name': 'West',
        'division': 8, 'division_name': 'Mountain',
    },
    '31': {
        'abbreviation': 'NE', 'name': 'Nebraska',
        'region': 2, 'region_name': 'Midwest',
        'division': 4, 'division_name': 'West North Central',
    },
    '32': {
        'abbreviation': 'NV', 'name': 'Nevada',
        'region': 4, 'region_name': 'West',
        'division': 8, 'division_name': 'Mountain',
    },
    '33': {
        'abbreviation': 'NH', 'name': 'New Hampshire',
        'region': 1, 'region_name': 'Northeast',
        'division': 1, 'division_name': 'New England',
    },
    '34': {
        'abbreviation': 'NJ', 'name': 'New Jersey',
        'region': 1, 'region_name': 'Northeast',
        'division': 2, 'division_name': 'Middle Atlantic',
    },
    '35': {
        'abbreviation': 'NM', 'name': 'New Mexico',
        'region': 4, 'region_name': 'West',
        'division': 8, 'division_name': 'Mountain',
    },
    '36': {
        'abbreviation': 'NY', 'name': 'New York',
        'region': 1, 'region_name': 'Northeast',
        'division': 2, 'division_name': 'Middle Atlantic',
    },
    '37': {
        'abbreviation': 'NC', 'name': 'North Carolina',
        'region': 3, 'region_name': 'South',
        'division': 5, 'division_name': 'South Atlantic',
    },
    '38': {
        'abbreviation': 'ND', 'name': 'North Dakota',
        'region': 2, 'region_name': 'Midwest',
        'division': 4, 'division_name': 'West North Central',
    },
    '39': {
        'abbreviation': 'OH', 'name': 'Ohio',
        'region': 2, 'region_name': 'Midwest',
        'division': 3, 'division_name': 'East North Central',
    },
    '40': {
        'abbreviation': 'OK', 'name': 'Oklahoma',
        'region': 3, 'region_name': 'South',
        'division': 7, 'division_name': 'West South Central',
    },
    '41': {
        'abbreviation': 'OR', 'name': 'Oregon',
        'region': 4, 'region_name': 'West',
        'division': 9, 'division_name': 'Pacific',
    },
    '42': {
        'abbreviation': 'PA', 'name': 'Pennsylvania',
        'region': 1, 'region_name': 'Northeast',
        'division': 2, 'division_name': 'Middle Atlantic',
    },
    '44': {
        'abbreviation': 'RI', 'name': 'Rhode Island',
        'region': 1, 'region_name': 'Northeast',
        'division': 1, 'division_name': 'New England',
    },
    '45': {
        'abbreviation': 'SC', 'name': 'South Carolina',
        'region': 3, 'region_name': 'South',
        'division': 5, 'division_name': 'South Atlantic',
    },
    '46': {
        'abbreviation': 'SD', 'name': 'South Dakota',
        'region': 2, 'region_name': 'Midwest',
        'division': 4, 'division_name': 'West North Central',
    },
    '47': {
        'abbreviation': 'TN', 'name': 'Tennessee',
        'region': 3, 'region_name': 'South',
        'division': 6, 'division_name': 'East South Central',
    },
    '48': {
        'abbreviation': 'TX', 'name': 'Texas',
        'region': 3, 'region_name': 'South',
        'division': 7, 'division_name': 'West South Central',
    },
    '49': {
        'abbreviation': 'UT', 'name': 'Utah',
        'region': 4, 'region_name': 'West',
        'division': 8, 'division_name': 'Mountain',
    },
    '50': {
        'abbreviation': 'VT', 'name': 'Vermont',
        'region': 1, 'region_name': 'Northeast',
        'division': 1, 'division_name': 'New England',
    },
    '51': {
        'abbreviation': 'VA', 'name': 'Virginia',
        'region': 3, 'region_name': 'South',
        'division': 5, 'division_name': 'South Atlantic',
    },
    '53': {
        'abbreviation': 'WA', 'name': 'Washington',
        'region': 4, 'region_name': 'West',
        'division': 9, 'division_name': 'Pacific',
    },
    '54': {
        'abbreviation': 'WV', 'name': 'West Virginia',
        'region': 3, 'region_name': 'South',
        'division': 5, 'division_name': 'South Atlantic',
    },
    '55': {
        'abbreviation': 'WI', 'name': 'Wisconsin',
        'region': 2, 'region_name': 'Midwest',
        'division': 3, 'division_name': 'East North Central',
    },
    '56': {
        'abbreviation': 'WY', 'name': 'Wyoming',
        'region': 4, 'region_name': 'West',
        'division': 8, 'division_name': 'Mountain',
    },
    '60': {
        'abbreviation': 'AS', 'name': 'America Samoa',
        'region': None, 'region_name': '',
        'division': None, 'division_name': '',
    },
    '66': {
        'abbreviation': 'GU', 'name': 'Guam',
        'region': None, 'region_name': '',
        'division': None, 'division_name': '',
    },
    '69': {
        'abbreviation': 'MP', 'name': 'Commonwealth of the Northern Mariana Islands',
        'region': None, 'region_name': '',
        'division': None, 'division_name': '',
    },
    '72': {
        'abbreviation': 'PR', 'name': 'Puerto Rico',
        'region': None, 'region_name': '',
        'division': None, 'division_name': '',
    },
    '78': {
        'abbreviation': 'VI', 'name': 'United States Virgin Islands',
        'region': None, 'region_name': '',
        'division': None, 'division_name': '',
    },
}

In [ ]:
def get_fips_code_for_state(state):
    for key, state_dict in STATE_FIPS_DICT.items():
        if state_dict['abbreviation'] == state.upper()\
        or state_dict['name'].upper() == state.upper():
            return key


STATE_ABBREV_LIST = sorted([
    state['abbreviation'] for fips, state in STATE_FIPS_DICT.items()
])

In [ ]:
state_number = get_fips_code_for_state(state)
if len(state)== 2:
    for key, state_dict in STATE_FIPS_DICT.items():
        if state_dict['abbreviation'] == state.upper():
            state = state_dict['name']

In [ ]:
GEO_TYPES_DICT = {
    'block group': {'level':'bg'},
    'tract': {'level':'ct'},
}

GEO_TYPES_LIST = sorted([
    key for key, value in GEO_TYPES_DICT.items()
])

In [ ]:
def get_geo_type():
    for key, value in GEO_TYPES_DICT.items():
        if value['level'] == geo_level.lower():
            return key
geo_type = get_geo_type()

if geo_type == None:
    print("invalid geographic level"+"\n"+ 'only "Census Tract" and "Block Group" are acceptable levels'+
         '\n'+ 'Please enter "CT" or "BG" as input parameters for the geo_type' )
    sys.exit()

In [ ]:
try:
    from six.moves.urllib import request as urllib2
except ImportError:
    import urllib2

FTP_HOME = 'ftp://ftp2.census.gov/geo/tiger/TIGER2012/'

In [ ]:
def get_filename_list_from_ftp(target, state):
    target_files = urllib2.urlopen(target).read().splitlines()
    filename_list = []

    for line in target_files:
        filename = '%s%s' % (target, line.decode().split()[-1])
        filename_list.append(filename)

    if state:
        state_check = '_%s_' % get_fips_code_for_state(state)
        filename_list = filter(
            lambda filename:
                state_check in filename or
                ('_us_' in filename and
                 '_us_zcta5' not in filename),
            filename_list
        )

    return filename_list

In [ ]:
def get_content_length(u):
    # u is returned by urllib2.urlopen
    if sys.version_info[0] == 2:
        return int(u.info().getheader("Content-Length"))
    else:
        return int(u.headers["Content-Length"])

In [ ]:
def download_files_in_list(filename_list, force=False):
    downloaded_filename_list = []
    for file_location in filename_list:
        filename = '%s/%s' % (DOWNLOAD_DIR, file_location.split('/')[-1])
        if force or not os.path.exists(filename):
            # Only download if required.
            u = urllib2.urlopen(file_location)
            f = open(filename, 'wb')
            file_size = get_content_length(u)

            print("Downloading: %s Bytes: %s" % (filename, file_size))
            file_size_dl = 0
            block_sz = 1000000
            while True:
                buffer = u.read(block_sz)
                if not buffer:
                    break

                file_size_dl += len(buffer)
                f.write(buffer)
                status = r"%10d  [%3.2f%%]" % (
                    file_size_dl, file_size_dl * 100. / file_size)
                status = status + chr(8) * (len(status) + 1)
                sys.stdout.write(status)
                sys.stdout.flush()

            f.close()
        downloaded_filename_list.append(filename)

    return downloaded_filename_list

In [ ]:
def extract_downloaded_file(filename, remove_on_error=True):
    zip_dir = filename.replace('.zip', '').split('/')[-1]
    target_dir = normpath(join(EXTRACT_DIR, zip_dir))

    print("Extracting: " + filename + " ...")
    try:
        zipped = zipfile.ZipFile(filename, 'r')
    except zipfile.BadZipFile as ze:
        if remove_on_error:
            os.remove(filename)
            raise Exception(
                "Removed corrupt zip file (%s). Retry download." % filename)
        raise ze

    zipped.extractall(target_dir)
    zipped.close()

In [ ]:
def get_one_geo_type(geo_type, state=None, year='2012'):
    if geo_type == 'block group':
        geo_type = 'bg'
    target = '%s%s/' % (FTP_HOME.replace('2012', year), geo_type.upper())

    print("Finding files in: " + target + " ...")
    filename_list = get_filename_list_from_ftp(target, state)
    downloaded_filename_list = download_files_in_list(filename_list)
     

    for filename in downloaded_filename_list:
        extract_downloaded_file(filename)
        
    return downloaded_filename_list

In [ ]:
zip_file_name = get_one_geo_type(geo_type, state, str(year))